<div style="background: linear-gradient(to right, #2b5876, #4e4376);" align="center">

# <span style="color: white; padding: 10px 20px; border-radius: 10px;">🔍 Segmentation Faciale par Deep Learning</span>

## <span s style="color: white;">Analyse Comparative des Architectures pour la Détection des Traits du Visage</span>
___

</div>

## *Downloading the LAPA Face Parsing Dataset*

In [ ]:
import kagglehub
import shutil
import os
# Download latest version
path = kagglehub.dataset_download("kiranraghavendrauci/lapa-face-parsing-dataset")
print("Path to dataset files:", path)

## *Building and Training Deep Learning Models for Semantic Segmentation*

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from glob import glob
from tensorflow.keras.layers import Conv2D, UpSampling2D, Conv2DTranspose, Concatenate, Input, Activation, BatchNormalization, Resizing, DepthwiseConv2D, GlobalAveragePooling2D, Dense, Lambda, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.mixed_precision import set_global_policy
from tensorflow.keras import backend as K

# Activer l'entraînement avec précision mixte
set_global_policy("mixed_float16")

# Réduire les logs TensorFlow
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Créer un répertoire si nécessaire
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Chargement et prétraitement des données
def load_dataset(path):
    train_x = sorted(glob(os.path.join(path, "train", "images", "*.jpg")))
    train_y = sorted(glob(os.path.join(path, "train", "labels", "*.png")))

    valid_x = sorted(glob(os.path.join(path, "val", "images", "*.jpg")))
    valid_y = sorted(glob(os.path.join(path, "val", "labels", "*.png")))

    return (train_x, train_y), (valid_x, valid_y)

def read_image_mask(x, y, image_h, image_w):
    """ Charger et redimensionner l'image et le masque """
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (image_w, image_h))
    x = x / 255.0
    x = x.astype(np.float32)

    y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    y = cv2.resize(y, (image_w, image_h))
    y = y.astype(np.int32)
    return x, y

def preprocess(x, y, image_h, image_w, num_classes):
    def f(x, y):
        x = x.decode()
        y = y.decode()
        return read_image_mask(x, y, image_h, image_w)

    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    mask = tf.one_hot(mask, num_classes)
    image.set_shape([image_h, image_w, 3])
    mask.set_shape([image_h, image_w, num_classes])
    return image, mask

def tf_dataset(X, Y, batch, image_h, image_w, num_classes):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.shuffle(buffer_size=5000).map(lambda x, y: preprocess(x, y, image_h, image_w, num_classes))
    ds = ds.batch(batch).prefetch(2)
    return ds

# Métriques d'évaluation
def iou(y_true, y_pred):
    y_true = K.flatten(K.argmax(y_true, axis=-1))
    y_pred = K.flatten(K.argmax(y_pred, axis=-1))
    intersection = K.sum(K.cast(K.equal(y_true, y_pred), dtype="float32"))
    union = K.sum(K.cast(K.not_equal(y_true, y_pred), dtype="float32"))
    return intersection / (union + intersection + K.epsilon())


def dice_coefficient(y_true, y_pred):
    y_true = K.flatten(K.argmax(y_true, axis=-1))
    y_pred = K.flatten(K.argmax(y_pred, axis=-1))
    numerator = 2 * K.sum(K.cast(K.equal(y_true, y_pred), dtype="float32"))
    denominator = K.sum(K.cast(K.not_equal(y_true, y_pred), dtype="float32")) + K.sum(K.cast(K.equal(y_true, y_pred), dtype="float32"))
    return numerator / (denominator + K.epsilon())

def precision(y_true, y_pred):
    y_true = K.flatten(K.argmax(y_true, axis=-1))
    y_pred = K.flatten(K.argmax(y_pred, axis=-1))
    true_positives = K.sum(K.cast(K.equal(y_true, y_pred), dtype="float32"))
    predicted_positives = K.sum(K.cast(K.greater(y_pred, 0), dtype="float32"))
    return true_positives / (predicted_positives + K.epsilon())

def recall(y_true, y_pred):
    y_true = K.flatten(K.argmax(y_true, axis=-1))
    y_pred = K.flatten(K.argmax(y_pred, axis=-1))
    true_positives = K.sum(K.cast(K.equal(y_true, y_pred), dtype="float32"))
    actual_positives = K.sum(K.cast(K.greater(y_true, 0), dtype="float32"))
    return true_positives / (actual_positives + K.epsilon())

# Définition des modèles
def build_unet(input_shape, num_classes):
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape)

    # Extraire les couches d'encodeur
    s1 = base_model.get_layer("block_1_expand_relu").output
    s2 = base_model.get_layer("block_3_expand_relu").output
    s3 = base_model.get_layer("block_6_expand_relu").output
    s4 = base_model.get_layer("block_13_expand_relu").output
    b1 = base_model.get_layer("block_16_project").output

    # Décodeur U-Net
    def decoder_block(inputs, skip, num_filters):
        x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
        x = Concatenate()([x, skip])
        x = Conv2D(num_filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Conv2D(num_filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Ajuster la taille de sortie pour qu'elle corresponde à l'entrée
    outputs = UpSampling2D(size=(2, 2))(d4)
    outputs = Conv2D(num_classes, 1, activation="softmax", padding="same")(outputs)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

def build_pspnet(input_shape, num_classes):
    base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape)
    
    # Encoder
    b1 = base_model.get_layer("block_16_project").output
    
    # Pyramid Pooling Module
    def psp_block(inputs, num_filters, pool_size):
        x = AveragePooling2D(pool_size=pool_size, strides=pool_size, padding='same')(inputs)
        x = Conv2D(num_filters, 1, padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        target_size = (inputs.shape[1], inputs.shape[2])
        x = UpSampling2D(size=(pool_size[0], pool_size[1]))(x)
        return x
    
    
    input_h = input_shape[0]
    feature_map_size = input_h // 32  
    
    pool_sizes = [
        (feature_map_size, feature_map_size),  # Pooling global
        (feature_map_size // 2, feature_map_size // 2),  # Taille 1/2
        (feature_map_size // 3, feature_map_size // 3),  # Taille 1/3
        (feature_map_size // 6, feature_map_size // 6)   # Taille 1/6
    ]
    
    # Module de Pooling Pyramidale
    pyramid_features = [b1]
    for pool_size in pool_sizes:
        pyramid_features.append(psp_block(b1, 256, pool_size))

    # Concaténer toutes les cartes de caractéristiques
    x = Concatenate()(pyramid_features)

    # Convolutions finales
    x = Conv2D(512, 3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Upsampling progressif pour correspondre aux dimensions d'entrée
    # Première upsampling (8x8 -> 16x16)
    x = Conv2DTranspose(256, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Deuxième upsampling (16x16 -> 32x32)
    x = Conv2DTranspose(256, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Troisième upsampling (32x32 -> 64x64)
    x = Conv2DTranspose(128, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Quatrième upsampling (64x64 -> 128x128)
    x = Conv2DTranspose(128, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Upsampling final (128x128 -> 256x256)
    x = Conv2DTranspose(64, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Ajustements finaux et sortie
    x = Conv2D(64, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model




def build_segnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Encoder
    def encoder_block(inputs, num_filters, pool_size=(2, 2)):
        x = Conv2D(num_filters, 3, padding="same", activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = Conv2D(num_filters, 3, padding="same", activation='relu')(x)
        x = BatchNormalization()(x)
        p = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=pool_size)(x)
        return x, p
    
    enc1, p1 = encoder_block(inputs, 64)
    enc2, p2 = encoder_block(p1, 128)
    enc3, p3 = encoder_block(p2, 256)
    enc4, p4 = encoder_block(p3, 512)
    enc5, p5 = encoder_block(p4, 512)
    
    # Decoder
    def decoder_block(inputs, skip, num_filters, up_size=(2, 2)):
        x = UpSampling2D(size=up_size)(inputs)
        x = Concatenate()([x, skip])
        x = Conv2D(num_filters, 3, padding="same", activation='relu')(x)
        x = BatchNormalization()(x)
        x = Conv2D(num_filters, 3, padding="same", activation='relu')(x)
        x = BatchNormalization()(x)
        return x
    
    dec1 = decoder_block(p5, enc5, 512)
    dec2 = decoder_block(dec1, enc4, 512)
    dec3 = decoder_block(dec2, enc3, 256)
    dec4 = decoder_block(dec3, enc2, 128)
    dec5 = decoder_block(dec4, enc1, 64)

    # Output
    outputs = Conv2D(num_classes, 1, activation="softmax", padding="same")(dec5)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Fonction d'entraînement
def train_model(model, train_ds, valid_ds, model_path, csv_path, num_epochs, lr):
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True, monitor="val_loss"),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
    ]

    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(lr),
        metrics=[iou, dice_coefficient, precision, recall]
    )

    model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=num_epochs,
        callbacks=callbacks
    )
    model.save(model_path)

def load_segmentation_models(model_dir):
  """Charge les modèles de segmentation à partir du répertoire spécifié."""
  unet_model = tf.keras.models.load_model(os.path.join(model_dir, "unet_model.keras"), custom_objects={'iou': iou, 'dice_coefficient': dice_coefficient, 'precision': precision, 'recall': recall})
  pspnet_model = tf.keras.models.load_model(os.path.join(model_dir, "pspnet_model.keras"), custom_objects={'iou': iou, 'dice_coefficient': dice_coefficient, 'precision': precision, 'recall': recall})
  segnet_model = tf.keras.models.load_model(os.path.join(model_dir, "segnet_model.keras"), custom_objects={'iou': iou, 'dice_coefficient': dice_coefficient, 'precision': precision, 'recall': recall})
 

  assert unet_model is not None, "Erreur lors du chargement du modèle U-Net."
  assert pspnet_model is not None, "Erreur lors du chargement du modèle PSPNet."
  assert segnet_model is not None, "Erreur lors du chargement du modèle SegNet."
  print("Tous les modèles sont correctement chargés.")

  return unet_model, pspnet_model, segnet_model


def load_and_preprocess_image(image_path, image_h=256, image_w=256):
    """Charge, redimensionne et normalise l'image."""
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (image_w, image_h))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

def display_mask(prediction, ax, cmap='viridis'):
    """Affiche le masque de segmentation."""
    mask = np.argmax(prediction, axis=-1)
    ax.imshow(mask, cmap=cmap)
    ax.axis('off')

def visualize_predictions(models, model_names, dataset_path, num_images=4, use_random_images=False):
    """Visualise les prédictions de plusieurs modèles."""
    if use_random_images:
      # Choisir aléatoirement des images
      all_image_paths = sorted(glob(os.path.join(dataset_path, "val", "images", "*.jpg")))
      test_image_paths = np.random.choice(all_image_paths, size=num_images, replace=False)
    else:
      # Utiliser les premières images du dossier
      test_image_paths = sorted(glob(os.path.join(dataset_path, "val", "images", "*.jpg")))[:num_images]

    # Affichage des prédictions
    fig, axes = plt.subplots(num_images, len(models) + 1, figsize=(5*(len(models)+1), 5 * num_images))
    fig.suptitle("Comparaison des prédictions des modèles", fontsize=16)

    for i, image_path in enumerate(test_image_paths):
        # Charger et prétraiter l'image
        image = load_and_preprocess_image(image_path)
        image = np.expand_dims(image, axis=0)  # Ajouter une dimension pour le batch
        
        # Afficher l'image originale
        axes[i, 0].imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
        axes[i, 0].axis('off')
        axes[i, 0].set_title("Image originale")
        
        # Faire la prédiction et afficher le résultat
        for j, (model, model_name) in enumerate(zip(models, model_names)):
            prediction = model.predict(image)
            display_mask(prediction[0], axes[i, j + 1])  # prediction[0] car predict retourne un batch de taille 1
            axes[i, j + 1].set_title(f"Masque prédit ({model_name})")

    plt.tight_layout()
    plt.show()

# Programme principal
if __name__ == "__main__":
    # Hyperparamètres
    image_h, image_w = 256, 256  # Réduction de la taille des images
    num_classes = 11
    batch_size = 8
    lr = 1e-4
    num_epochs = 10

    # Chemins
    dataset_path = "/kaggle/input/lapa-face-parsing-dataset/LaPa"
    create_dir("files")

    # Chargement du dataset
    (train_x, train_y), (valid_x, valid_y) = load_dataset(dataset_path)

    # Création des datasets TensorFlow
    train_ds = tf_dataset(train_x, train_y, batch_size, image_h, image_w, num_classes)
    valid_ds = tf_dataset(valid_x, valid_y, batch_size, image_h, image_w, num_classes)


    # Modèles
    models = {
        "unet": build_unet,
        "pspnet": build_pspnet,
        "segnet": build_segnet,
    }

    # Entraînement de chaque modèle
    for model_name, model_builder in models.items():
        print(f"Entraînement de {model_name}...")
        model = model_builder((image_h, image_w, 3), num_classes)
        model_path = os.path.join("files", f"{model_name}_model.keras")
        csv_path = os.path.join("files", f"{model_name}_data.csv")
        train_model(model, train_ds, valid_ds, model_path, csv_path, num_epochs, lr)

## *Visualisation des Prédictions des Modèles de Segmentation Faciale*

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from glob import glob

def load_and_preprocess_image(image_path, image_h=256, image_w=256):
    """Charge, redimensionne et normalise l'image."""
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (image_w, image_h))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

def display_mask(prediction, ax, cmap='viridis', num_classes=11):
    """Affiche le masque de segmentation avec une palette de couleurs adaptée."""
    mask = np.argmax(prediction, axis=-1)
    
    # Créer une palette de couleurs personnalisée pour les différentes parties du visage
    colors = plt.cm.get_cmap('tab20')(np.linspace(0, 1, num_classes))
    custom_cmap = plt.cm.colors.ListedColormap(colors)
    
    im = ax.imshow(mask, cmap=custom_cmap, vmin=0, vmax=num_classes-1)
    ax.axis('off')
    return im

def visualize_predictions(models, model_names, image_paths, save_path=None):
    """
    Visualise les prédictions de plusieurs modèles.
    
    Args:
        models: Liste des modèles à comparer
        model_names: Liste des noms des modèles
        image_paths: Liste des chemins d'images à tester
        save_path: Chemin où sauvegarder les visualisations (optionnel)
    """
    num_images = len(image_paths)
    
    # Création de la figure avec une taille adaptée
    fig, axes = plt.subplots(num_images, len(models) + 1, 
                            figsize=(5*(len(models)+1), 5 * num_images))
    fig.suptitle("Comparaison des prédictions des modèles de segmentation", 
                 fontsize=16, y=1.02)
    
    # Ajuster axes pour une seule image
    if num_images == 1:
        axes = axes.reshape(1, -1)

    # Pour chaque image
    for i, image_path in enumerate(image_paths):
        # Charger et prétraiter l'image
        image = load_and_preprocess_image(image_path)
        image_batch = np.expand_dims(image, axis=0)
        
        # Afficher l'image originale
        axes[i, 0].imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
        axes[i, 0].axis('off')
        axes[i, 0].set_title("Image originale", pad=10)
        
        # Faire la prédiction avec chaque modèle
        for j, (model, model_name) in enumerate(zip(models, model_names)):
            prediction = model.predict(image_batch, verbose=0)
            im = display_mask(prediction[0], axes[i, j + 1])
            axes[i, j + 1].set_title(f"{model_name}", pad=10)
    
    # Ajuster l'espacement
    plt.tight_layout()
    
    # Ajouter une colorbar pour la légende
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
    cbar = plt.colorbar(im, cax=cbar_ax)
    cbar.set_label('Classes de segmentation', rotation=270, labelpad=15)
    
    # Sauvegarder si un chemin est spécifié
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
        print(f"Visualisation sauvegardée dans {save_path}")
    
    plt.show()

# Fonction pour charger les modèles
def load_segmentation_models(model_dir):
    """Charge les modèles de segmentation à partir du répertoire spécifié."""
    # Définir les objets personnalisés pour le chargement
    custom_objects = {
        'iou': iou,
        'dice_coefficient': dice_coefficient,
        'precision': precision,
        'recall': recall
    }
    
    # Charger les modèles
    unet_model = tf.keras.models.load_model(
        os.path.join(model_dir, "unet_model.keras"),
        custom_objects=custom_objects
    )
    
    pspnet_model = tf.keras.models.load_model(
        os.path.join(model_dir, "pspnet_model.keras"),
        custom_objects=custom_objects
    )
    
    segnet_model = tf.keras.models.load_model(
        os.path.join(model_dir, "segnet_model.keras"),
        custom_objects=custom_objects
    )

    print("Tous les modèles sont correctement chargés.")
    return unet_model, pspnet_model, segnet_model

# Code principal pour l'utilisation
def main():
    model_dir = "files"
    dataset_path = "/kaggle/input/lapa-face-parsing-dataset/LaPa" 
    # Charger les modèles
    unet_model, pspnet_model, segnet_model = load_segmentation_models(model_dir)
    models = [unet_model, pspnet_model, segnet_model]
    model_names = ["U-Net", "PSPNet", "SegNet"]
    
    test_images = sorted(glob(os.path.join(dataset_path, "val", "images", "*.jpg")))[5:6]
    
    results_dir = "resultats_segmentation"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
    
    # Visualiser les prédictions
    visualize_predictions(
        models,
        model_names,
        test_images,
        save_path=os.path.join(results_dir, "comparaison_modeles.png") # Sauvegarder l'image
    )

if __name__ == "__main__":
    main()